### Association rules Department -Pantry
#### Department - Pantry

In [37]:
##importing libraries
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import OnehotTransactions
import pandas as pd
from sklearn import cluster, datasets
import numpy as np
import scipy
from scipy.cluster.vq import kmeans,vq
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt

In [38]:
# Csv contains data of trainorders and prior orders
combine_train_prior=pd.read_csv("CombineOrders1.csv",encoding = "ISO-8859-1")

In [39]:
## altering datatypes for ease of use
combine_train_prior['aisle_id']=combine_train_prior['aisle_id'].astype('int16')
combine_train_prior['order_id']=combine_train_prior['order_id'].astype('int32')
combine_train_prior['product_id']=combine_train_prior['product_id'].astype('int32')
combine_train_prior['add_to_cart_order']=combine_train_prior['add_to_cart_order'].astype('int16')
combine_train_prior['reordered']=combine_train_prior['reordered'].astype('int8')
combine_train_prior['department_id']=combine_train_prior['department_id'].astype('int8')



In [40]:
combine_train_prior['product_name'] = combine_train_prior['product_name'].str.strip()
combine_train_prior.dropna(axis=0, subset=['order_id'], inplace=True)

In [41]:
## this will give us the popular departments as departments with the highest value counts will be those from where consumers have purchased the most
##Top 5 Departments are produce,dairy eggs, snacks, beverages, frozen
combine_train_prior['department'].value_counts()

produce            9888378
dairy eggs         5631067
snacks             3006412
beverages          2804175
frozen             2336858
pantry             1956819
bakery             1225181
canned goods       1114857
deli               1095540
dry goods pasta     905340
household           774652
meat seafood        739238
breakfast           739069
personal care       468693
babies              438743
international       281155
alcohol             159294
pets                102221
missing              77396
other                38086
bulk                 35932
Name: department, dtype: int64

In [42]:
combine_train_prior

,Unnamed: 0,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,aisle,department
0,0,1,49302,1,1,112108,train,4,4,10,9.0,Bulgarian Yogurt,120,16,yogurt,dairy eggs
1,1,1,43633,5,1,112108,train,4,4,10,9.0,Lightly Smoked Sardines in Olive Oil,95,15,canned meat seafood,canned goods
2,2,1,10246,3,0,112108,train,4,4,10,9.0,Organic Celery Hearts,83,4,fresh vegetables,produce
3,3,1,22035,8,1,112108,train,4,4,10,9.0,Organic Whole String Cheese,21,16,packaged cheese,dairy eggs
4,4,1,47209,7,0,112108,train,4,4,10,9.0,Organic Hass Avocado,24,4,fresh fruits,produce
5,5,1,13176,6,0,112108,train,4,4,10,9.0,Bag of Organic Bananas,24,4,fresh fruits,produce
6,6,1,11109,2,1,112108,train,4,4,10,9.0,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,other creams cheeses,dairy eggs
7,7,1,49683,4,0,112108,train,4,4,10,9.0,Cucumber Kirby,83,4,fresh vegetables,produce
8,8,36,46979,8,1,79431,train,23,6,18,30.0,Asparagus,83,4,fresh vegetables,produce
9,9,36,39612,1,0,79431,train,23,6,18,30.0,Grated Pecorino Romano Cheese,2,16,specialty cheeses,dairy eggs


In [43]:
## Using Apriori we will find if there are association rules found for products bought from department Dairy Eggs
dept_produce=combine_train_prior[combine_train_prior['department'] =="pantry"].reset_index()

In [44]:
## Preparing dataset for Apriori
temp_df=dept_produce.loc[:,['order_id','product_name']]

In [67]:
## In order to find association we have considered those orders that have products which 4 or more in number
grouped_data=temp_df.groupby("order_id").filter(lambda x: len(x) >=4)

In [46]:
grouped_data

,order_id,product_name
1,98,Organic Ketchup
2,98,Organic Corn Starch
3,98,100% Organic Unbleached All-Purpose Flour
4,98,Organic Extra Virgin Oil Olive
25,1620,Organic Hot Salsa
26,1620,Rosemary
27,1620,Arrowroot
28,1620,of Norwich Original English Mustard Powder Dou...
29,1620,Active Dry Yeast
30,1620,Ground Allspice


In [68]:
## Pivoting the table to get occurences of a product in a particuler order
from pandas import *
# table=pd.DataFrame()
table1=pivot_table(grouped_data,index='order_id',columns='product_name',aggfunc=len)

In [69]:
#relace NAN with 0
table1=table1.fillna(0)

In [70]:
## of the product exists then value changed to 1 else 0
def encode_units(x):
    if x == 0.0:
        return 0
    if x >= 1.0:
        return 1


In [71]:
basket_sets = table1.applymap(encode_units)

In [51]:
basket_sets

product_name,& Go! Hazelnut Spread + Pretzel Sticks,1 to 1 Gluten Free Baking Flour,"1-to-1 Baking Flour, Gluten/Wheat/Dairy Free",100% All Natural Creamy Peanut Butter,100% All Natural Crunchy Peanut Butter,100% Cacao Cocoa Powder,100% Cacao Natural Unsweetened,100% Cacao Unsweetened Chocolate Baking Bar,100% Canola Oil,100% Canola Oil Original No-Stick Spray,...,Zesty Italian Dressing,Zesty Italian Dressing & Recipe Mix,Zesty Italian Salad Dressing & Recipe Mix,Zesty Ranch Dressing,Zinger Babies,Zoe Organic EVOO,of Norwich Original English Mustard Powder Double Superfine,with Lime Juice Mayonesa Mayonnaise,with Olive Oil Mayonnaise,with Olive Oil Mayonnaise Dressing
order_id,,,,,,,,,,,,,,,,,,,,,
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
195,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
256,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
317,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
325,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
329,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
400,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [72]:
## Running Apriori to get frequent itemset with a minimum suppirt of 0.01. ie the combination must exist atleast in 1% of dataset
frequent_itemsets = apriori(table1, min_support=0.01, use_colnames=True)

In [73]:
frequent_itemsets

,support,itemsets
0,0.023617,[100% Pure Pumpkin]
1,0.010879,[60% Cacao Bittersweet Premium Baking Chips]
2,0.017613,[Active Dry Yeast]
3,0.027474,[All Purpose Flour]
4,0.017638,[Almond Meal/Flour]
5,0.022700,[Apple Cider Vinegar]
6,0.010125,[Apricot Preserves]
7,0.019849,[Baking Powder]
8,0.025376,[Cane Sugar]
9,0.023605,[Canola Oil]


In [74]:
# Getting the rules of association based on the minimum support selected
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

In [75]:
rules

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction


In [63]:
## Since, ideally Lift must greater than 1 , we have selected the rules to have Lift to be greater than 1, confidence greater than 30% and a conviction of 40%
final_result=rules[ (rules['lift'] > 1) &
       (rules['confidence'] >= 0.3) & rules['conviction']<1.4 ]
final_result

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction


In [76]:
frequent_itemsets.to_csv("Department_PantryApriori.csv",sep=',')


In [77]:
test1=pd.read_csv("Department_PantryApriori.csv")

In [78]:
test1

,Unnamed: 0,support,itemsets
0,0,0.023617,[u'100% Pure Pumpkin']
1,1,0.010879,[u'60% Cacao Bittersweet Premium Baking Chips']
2,2,0.017613,[u'Active Dry Yeast']
3,3,0.027474,[u'All Purpose Flour']
4,4,0.017638,[u'Almond Meal/Flour']
5,5,0.022700,[u'Apple Cider Vinegar']
6,6,0.010125,[u'Apricot Preserves']
7,7,0.019849,[u'Baking Powder']
8,8,0.025376,[u'Cane Sugar']
9,9,0.023605,[u'Canola Oil']


In [79]:
## Cleaning the resultset for usability
test1=test1.replace({'frozenset': ''}, regex=True)

In [80]:
test1=test1.replace({"u'": ''}, regex=True)

In [81]:
test1

,Unnamed: 0,support,itemsets
0,0,0.023617,[100% Pure Pumpkin']
1,1,0.010879,[60% Cacao Bittersweet Premium Baking Chips']
2,2,0.017613,[Active Dry Yeast']
3,3,0.027474,[All Purpose Flour']
4,4,0.017638,[Almond Meal/Flour']
5,5,0.022700,[Apple Cider Vinegar']
6,6,0.010125,[Apricot Preserves']
7,7,0.019849,[Baking Powder']
8,8,0.025376,[Cane Sugar']
9,9,0.023605,[Canola Oil']


In [83]:
test1['itemsets'] = test1['itemsets'].str.strip('()').str.strip('[]')
test1['itemsets'] = test1['itemsets'].str.strip('[]')
test1['itemsets'] = test1['itemsets'].str.strip("'")
test1['itemsets'] = test1['itemsets'].str.replace("'",'')

In [ ]:
# test1['consequents'] = test1['consequents'].str.strip('()')
# test1['consequents'] = test1['consequents'].str.strip('[]')
# test1['consequents'] = test1['consequents'].str.strip("'")
# test1['consequents'] = test1['consequents'].str.replace("'",'')

In [84]:
test1=test1.drop(['Unnamed: 0'], axis = 1)

In [85]:
test1

,support,itemsets
0,0.023617,100% Pure Pumpkin
1,0.010879,60% Cacao Bittersweet Premium Baking Chips
2,0.017613,Active Dry Yeast
3,0.027474,All Purpose Flour
4,0.017638,Almond Meal/Flour
5,0.022700,Apple Cider Vinegar
6,0.010125,Apricot Preserves
7,0.019849,Baking Powder
8,0.025376,Cane Sugar
9,0.023605,Canola Oil


In [ ]:
## Writing results to a csv file
test1.to_csv("FinalDepartment_PantryApriori.csv")

In [ ]:
test1

In [ ]:
## testing the result set
t=test1[test1['antecedants'] =="Organic Strawberries, Organic Baby Spinach"].reset_index()